In [1]:
# === Imports and config ===
import os
import warnings
import math
import time
import glob
import json
import gc
from collections import defaultdict

import numpy as np
import pandas as pd

import pvlib
from pvlib.location import Location
from pvlib.modelchain import ModelChain
from pvlib.pvsystem import PVSystem
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

# Rutas principales
MASTER_CSV = "solar_weather_all_cities.csv"   # generated in phase 1
RESULTS_DIR = "results"                       #  .pkl per city with AC/DC ratios series
RESULTS_SUMMARY = "results_summary.csv"       # yearly summary per city/ILR (append)

os.makedirs(RESULTS_DIR, exist_ok=True)

# Control
CHUNKSIZE = 250_000       # rows per chunk to read master CSV without running out of RAM
TZ = "UTC"                # Timezone
PRINT_EVERY = 1           # Simple logging

In [2]:
# === Celda 2: Coordinates and plant configuration ===

# Build coord_df reading only city, lat, lon from master CSV
def build_coord_df_from_master(master_csv, chunksize=250_000):
    cols = ["city", "lat", "lon"]
    seen = {}
    for chunk in pd.read_csv(master_csv, usecols=cols, chunksize=chunksize):
        # Keep first occurrence of city
        for city, lat, lon in zip(chunk["city"], chunk["lat"], chunk["lon"]):
            if city not in seen:
                seen[city] = (lat, lon)
    if not seen:
        raise ValueError(f"No se encontraron ciudades en {master_csv}")

    df = pd.DataFrame(
        [(c, v[0], v[1]) for c, v in seen.items()],
        columns=["city", "latitude", "longitude"]
    ).set_index("city").sort_index()
    return df

# Create coord_df from master CSV
coord_df = build_coord_df_from_master(MASTER_CSV)
display(coord_df.head())

# Silence warning duplicate from pvlib
warnings.filterwarnings(
    "ignore",
    message=".*duplicate.*",
    category=UserWarning,
    module="pvlib.pvsystem"
)

# PV plant configuration
def plant_configuration():
    """
    common Config for all locations (NREL CEC).
    Returns:
      - df_ILR_power: Power and ILR table
      - inverter: inverter values
      - module: Module values
    """
    url_inverter = 'https://raw.githubusercontent.com/NREL/SAM/develop/deploy/libraries/CEC%20Inverters.csv'
    url_modules  = 'https://raw.githubusercontent.com/NREL/SAM/patch/deploy/libraries/CEC%20Modules.csv'

    cec_inverter = pvlib.pvsystem.retrieve_sam(path=url_inverter)
    cec_modules  = pvlib.pvsystem.retrieve_sam(path=url_modules)

    # Equipment selection
    inverter = cec_inverter['Chint_Power_Systems_America__CPS_SCH350KTL_DO_US_800__800V_']
    module   = cec_modules['Chint_New_Energy_Technology_Co__Ltd__CHSM66M_DG__F_BH_635']

    # Config del sistema
    n_inv = 143
    n_modules_per_strings = 26
    n_strings = np.arange(21.2, 60.3, 1.0)

    P_AC_inverter_MVA  = inverter.Paco
    P_DC_module_STC     = module.STC

    P_AC_output_power_inverter_MVA = n_inv * P_AC_inverter_MVA / 1e6
    P_DC_input_power_inverter_MWp  = n_inv * P_DC_module_STC * n_modules_per_strings * n_strings / 1e6
    ILR = P_DC_input_power_inverter_MWp / P_AC_output_power_inverter_MVA

    df_ILR_power = pd.DataFrame({
        'ILR': np.round(ILR, 2),
        'P_DC_input_power_inverter_MWp': np.round(P_DC_input_power_inverter_MWp, 2),
        'P_AC_output_power_inverter_MVA': P_AC_output_power_inverter_MVA,
        'num_inversores': n_inv,
        'num_strings': n_strings,
        'num_modules_per_string': n_modules_per_strings
    })

    return df_ILR_power, inverter, module

# Prepare config
temperature_parameters = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']
df_ILR_power, inverter, module = plant_configuration()

#Output variables
ILR_round = df_ILR_power['ILR'].values
P_DC_input_power_inverter_MWp = df_ILR_power['P_DC_input_power_inverter_MWp'].values
P_AC_output_power_inverter_MVA = df_ILR_power['P_AC_output_power_inverter_MVA'].iloc[0]


,latitude,longitude
city,,
Amsterdam,52.36760,4.904100
Athens,37.98380,23.727500
Berlin,52.52281,13.408176
Bratislava,48.14860,17.107700
Brussels,50.85030,4.351700


In [3]:
# === Save Plant configuration ===
import json

CONFIG_DIR = "plant_config"
os.makedirs(CONFIG_DIR, exist_ok=True)

# Tables -> CSV
coord_df.to_csv(os.path.join(CONFIG_DIR, "coord_df.csv"))                    # city, latitude, longitude
df_ILR_power.to_csv(os.path.join(CONFIG_DIR, "df_ILR_power.csv"), index=False)

# Series -> JSON 
with open(os.path.join(CONFIG_DIR, "inverter.json"), "w", encoding="utf-8") as f:
    json.dump(inverter.to_dict(), f, indent=2, default=float)
with open(os.path.join(CONFIG_DIR, "module.json"), "w", encoding="utf-8") as f:
    json.dump(module.to_dict(), f, indent=2, default=float)

# Temperature model parameters -> JSON
with open(os.path.join(CONFIG_DIR, "temperature_parameters.json"), "w", encoding="utf-8") as f:
    json.dump(temperature_parameters, f, indent=2, default=float)

# Configuration output
meta = {
    "P_AC_output_power_inverter_MW": float(P_AC_output_power_inverter_MVA),
    "n_inv": int(df_ILR_power["num_inversores"].iloc[0]),
    "n_modules_per_string": int(df_ILR_power["num_modules_per_string"].iloc[0]),
    "strings_min": float(df_ILR_power["num_strings"].min()),
    "strings_max": float(df_ILR_power["num_strings"].max()),
    "inverter_key": getattr(inverter, "name", None),
    "module_key": getattr(module, "name", None),
}
with open(os.path.join(CONFIG_DIR, "meta.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2, default=float)

print("✔ Configuración guardada en:", os.path.abspath(CONFIG_DIR))

✔ Configuración guardada en: C:\Users\danie\PycharmProjects\solarPV\plant_config


In [41]:
# === Multipurpose functions ===

def validate_master(master_csv=MASTER_CSV, chunksize=CHUNKSIZE):
    """ Validate columns and parse 'time' """
    required = {"time","ghi","dni","dhi","wind_speed","temp_air","city","lat","lon"}
    try:
        hdr = pd.read_csv(master_csv, nrows=1)
    except Exception as e:
        raise RuntimeError(f"No se pudo leer {master_csv}: {e}")
    missing = required - set(hdr.columns)
    if missing:
        raise ValueError(f"Faltan columnas en {master_csv}: {sorted(missing)}")
    # Time parse
    for ch in pd.read_csv(master_csv, usecols=list(required), chunksize=chunksize):
        t = pd.to_datetime(ch["time"], utc=True, errors="coerce")
        if t.isna().any():
            raise ValueError("Hay filas con 'time' no parseable a datetime.")
        break
    print(f"✔ {master_csv} validado (muestra OK).")

def city_results_path(city):
    """Results Filepath per city (pickle)."""
    safe_city = city.replace(" ", "_")
    return os.path.join(RESULTS_DIR, f"{safe_city}_results.pkl")

def city_results_exists(city):
    return os.path.exists(city_results_path(city))

def save_city_results(city, time_index, ac_list, dc_list, ilr_values):
    """
    Save results per city:
      - time_index: DatetimeIndex
      - ac_list / dc_list: lista of lists (une per ILR)
      - ilr_values: array/list with ILR values
    """
    df_out = pd.DataFrame(index=time_index)
    # AC_i y DC_i por ILR; nombres legibles
    for i, ilr in enumerate(ilr_values):
        df_out[f"AC_MW_ILR_{ilr}"] = ac_list[i].values
        df_out[f"DC_MW_ILR_{ilr}"] = dc_list[i].values
    df_out["city"] = city
    df_out.to_pickle(city_results_path(city))

def append_summary(city, time_index, ac_list, ilr_values):
    """
    Write/add yearly summary per ciudad and ILR to RESULTS_SUMMARY:
      columns: city, ILR, E_AC_MWh (minute-level aggregation /60), P_AC_nom_MVA
    """
    rows = []
    for i, ilr in enumerate(ilr_values):
        e_mwh = ac_list[i].sum() / 60.0  # MW-min → MWh
        rows.append({
            "city": city,
            "ILR": ilr,
            "E_AC_MWh": float(e_mwh),
            "P_AC_nom_MVA": float(P_AC_output_power_inverter_MVA)
        })
    df_sum = pd.DataFrame(rows)
    header = not os.path.exists(RESULTS_SUMMARY)
    df_sum.to_csv(RESULTS_SUMMARY, mode="a", header=header, index=False)

def load_city_from_master(city, path=MASTER_CSV, chunksize=CHUNKSIZE):
    """
    Read from the master CSV only the rows for 'city', in chunks (memory-efficient).
Return a DataFrame with a UTC datetime index and the following columns:
ghi, dni, dhi, wind_speed, temp_air, lat, lon, city.
    """
    parts = []
    cols_needed = ["time", "ghi", "dni", "dhi", "wind_speed", "temp_air", "city", "lat", "lon"]
    for i, chunk in enumerate(pd.read_csv(path, usecols=cols_needed, chunksize=chunksize)):
        sub = chunk.loc[chunk["city"] == city]
        if not sub.empty:
            # parse time to datetime with tz UTC
            t = pd.to_datetime(sub["time"], utc=True, errors="coerce")
            sub = sub.assign(time=t).dropna(subset=["time"])
            parts.append(sub[cols_needed])
        if (i+1) % PRINT_EVERY == 0:
            pass  # logging optional

    if not parts:
        raise ValueError(f"No se encontraron datos para la ciudad '{city}' en {path}")

    df = pd.concat(parts, ignore_index=True)
    df = df.sort_values("time").set_index("time")

    # tz UTC
    if df.index.tz is None:
        df.index = df.index.tz_localize(TZ)
    else:
        df.index = df.index.tz_convert(TZ)

    # Floats Downcast for memory savings
    for col in ["ghi","dni","dhi","wind_speed","temp_air","lat","lon"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce", downcast="float")

    return df

In [42]:
# === Celda 4: Simulación de una ciudad ===

def simulate_city(city, ilr_rows=None, write_summary=True, overwrite=False):
    """
    Run the full simulation for 'city'.

    ilr_rows: iterable of df_ILR_power indices (to limit ILR range if desired)

    write_summary: if True, append the annual summary to RESULTS_SUMMARY

    overwrite: if False and the city's .pkl file already exists, skip it.
    """
    if not overwrite and city_results_exists(city):
        print(f"⏭️  Saltando {city}: resultados ya existen ({city_results_path(city)})")
        return

    if city not in coord_df.index:
        raise ValueError(f"Coordenadas no encontradas para '{city}'")

    lat = coord_df.loc[city, 'latitude']
    lon = coord_df.loc[city, 'longitude']
    tilt = lat * 0.76 + 3.1
    surface_azimuth = 180

    print(f"📦 Cargando meteo+irradiancia de {city} desde {MASTER_CSV} ...")
    solar_weather_df = load_city_from_master(city)  # índice datetime tz-aware

    # Chequeo defensivo de columnas para pvlib
    need = {"ghi","dni","dhi","temp_air","wind_speed"}
    missing = need - set(solar_weather_df.columns)
    if missing:
        raise ValueError(f"Faltan columnas para pvlib en {city}: {sorted(missing)}")

    # pvlib awaits index tz-aware
    if solar_weather_df.index.tz is None:
        solar_weather_df.index = solar_weather_df.index.tz_localize(TZ)
    else:
        solar_weather_df.index = solar_weather_df.index.tz_convert(TZ)

    # Location
    site_location = Location(latitude=lat, longitude=lon, tz=TZ, altitude=0, name=city)

    # IlR range
    rows_to_sim = df_ILR_power.index if ilr_rows is None else ilr_rows

    ac_list = []
    dc_list = []
    ilr_vals = []

    for i in rows_to_sim:
        row = df_ILR_power.loc[i]
        strings = row['num_strings']
        n_inv = int(row['num_inversores'])
        n_modules_per_string = int(row['num_modules_per_string'])
        ilr = float(row['ILR'])

        system = PVSystem(
            surface_tilt=tilt,
            surface_azimuth=surface_azimuth,
            module_parameters=module,
            inverter_parameters=inverter,
            modules_per_string=n_modules_per_string,
            strings_per_inverter=strings,
            temperature_model_parameters=temperature_parameters
        )

        modelchain = ModelChain(system=system, location=site_location,
                                aoi_model='no_loss', spectral_model='no_loss')

        # Run model in Dataframe
        modelchain.run_model(solar_weather_df)

        # AC and DC Power [MW]
        ac = n_inv * modelchain.results.ac / 1e6
        ac = ac.clip(lower=0)  # no negative values
        dc = n_inv * modelchain.results.dc['p_mp'] / 1e6
        dc = dc.clip(lower=0)

        ac_list.append(ac)
        dc_list.append(dc)
        ilr_vals.append(ilr)

        # Cleaning
        del modelchain
        gc.collect()

    # Incremental saving
    save_city_results(city, solar_weather_df.index, ac_list, dc_list, ilr_vals)
    if write_summary:
        append_summary(city, solar_weather_df.index, ac_list, ilr_vals)

    print(f"✅ Simulación completada: {city}  |  ILRs simulados: {len(ilr_vals)}")

In [43]:
# === Celda 5: Ejecución por lotes ===

def run_model_batch(target_cities=None, ilr_rows=None, overwrite=False):
    """
    Run batch simulations:

    target_cities: list of cities (if None, take all cities from coord_df that are present in MASTER_CSV)

    ilr_rows: iterable of df_ILR_power indices to simulate (for quick tests)

    overwrite: if False, do not recalculate cities that already have a .pkl created.
    """
    if target_cities is None:
        # Check cities in master file
        try:
            cities_in_master = set()
            for chunk in pd.read_csv(MASTER_CSV, usecols=["city"], chunksize=CHUNKSIZE):
                cities_in_master.update(chunk["city"].unique().tolist())
            target_cities = sorted(list(set(coord_df.index).intersection(cities_in_master)))
        except Exception as e:
            print(f"⚠️ No pude inferir ciudades del maestro: {e}")
            target_cities = list(coord_df.index)

    for c in target_cities:
        try:
            simulate_city(c, ilr_rows=ilr_rows, write_summary=True, overwrite=overwrite)
        except Exception as e:
            print(f"❌ Error en {c}: {e}")

In [44]:
# === Run examples ===

# 1) Validate master before running simulations
# validate_master()

# 2) Test one city with a few ILR rows (fast)
# run_model_batch(target_cities=["Madrid"], ilr_rows=df_ILR_power.index[:3])

# 3) Run a full city (all ILRs), resumable
# run_model_batch(target_cities=["Madrid"])

# 4) Run several cities in batches (resumable; skips if .pkl already exists)
# run_model_batch(target_cities=["Madrid", "Paris", "Berlin"])

# 5) Force recalculation of a city (overwrite)
# run_model_batch(target_cities=["Madrid"], overwrite=True)

# 6) Run all cities present in the master CSV:
# run_model_batch()


In [45]:
# === sanity check ===

# 1) Validate master
validate_master()

✔ solar_weather_all_cities.csv validado (muestra OK).


In [29]:
# 2) Test city load
city_test = "Madrid"
df_test = load_city_from_master(city_test)
print(city_test, "| filas:", len(df_test), "| rango:", df_test.index.min(), "→", df_test.index.max())
print("cols:", sorted(df_test.columns.tolist())[:10], "...")

Madrid | filas: 527040 | rango: 2024-01-01 00:00:00+00:00 → 2024-12-31 23:59:00+00:00
cols: ['city', 'dhi', 'dni', 'ghi', 'lat', 'lon', 'temp_air', 'wind_speed'] ...


In [30]:
# 3) Simulate 2-3 ILR for testing purposes
run_model_batch(target_cities=[city_test], ilr_rows=df_ILR_power.index[:3])

📦 Cargando meteo+irradiancia de Madrid desde solar_weather_all_cities.csv ...
✅ Simulación completada: Madrid  |  ILRs simulados: 3


In [46]:
run_model_batch()

📦 Cargando meteo+irradiancia de Amsterdam desde solar_weather_all_cities.csv ...
✅ Simulación completada: Amsterdam  |  ILRs simulados: 40
📦 Cargando meteo+irradiancia de Athens desde solar_weather_all_cities.csv ...
✅ Simulación completada: Athens  |  ILRs simulados: 40
📦 Cargando meteo+irradiancia de Berlin desde solar_weather_all_cities.csv ...
✅ Simulación completada: Berlin  |  ILRs simulados: 40
📦 Cargando meteo+irradiancia de Bratislava desde solar_weather_all_cities.csv ...
✅ Simulación completada: Bratislava  |  ILRs simulados: 40
📦 Cargando meteo+irradiancia de Brussels desde solar_weather_all_cities.csv ...
✅ Simulación completada: Brussels  |  ILRs simulados: 40
📦 Cargando meteo+irradiancia de Bucharest desde solar_weather_all_cities.csv ...
✅ Simulación completada: Bucharest  |  ILRs simulados: 40
📦 Cargando meteo+irradiancia de Budapest desde solar_weather_all_cities.csv ...
✅ Simulación completada: Budapest  |  ILRs simulados: 40
📦 Cargando meteo+irradiancia de Copenhage